In [49]:
import tushare as ts
ts.set_token('8df6b825b87018ded5c09457e749cd1430d60febc9ab45d699eb0aab')
pro = ts.pro_api()

In [50]:
# Start get stock info
stockBasic = pro.query('stock_basic')
print(stockBasic)

        ts_code  symbol      name area industry market list_date
0     000001.SZ  000001      平安银行   深圳       银行     主板  19910403
1     000002.SZ  000002       万科A   深圳     全国地产     主板  19910129
2     000004.SZ  000004      国华网安   深圳      互联网     主板  19910114
3     000005.SZ  000005      世纪星源   深圳     环境保护     主板  19901210
4     000006.SZ  000006      深振业A   深圳     区域地产     主板  19920427
...         ...     ...       ...  ...      ...    ...       ...
4169  688777.SH  688777      中控技术   浙江     软件服务    科创板  20201124
4170  688788.SH  688788      科思科技   深圳     通信设备    科创板  20201022
4171  688819.SH  688819      天能股份   浙江     电气设备    科创板  20210118
4172  688981.SH  688981    中芯国际-U   上海      半导体    科创板  20200716
4173  689009.SH  689009  九号公司-UWD   北京     专用机械    CDR  20201029

[4174 rows x 7 columns]


In [51]:
#write into HDFS
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
spark = SparkSession.builder.appName("getData").enableHiveSupport().getOrCreate()
stockBasicDF = spark.createDataFrame(stockBasic)
stockBasicDF.write.mode('overwrite').parquet("hdfs://localhost:9999/user/stock/stock_basic")

In [52]:
PA_basic = pro.query("stock_company",ts_code = '601318.SH', fields="ts_code, exchange, reg_capital, province, city, introduction, employees, main_business, business_scope")
print(PA_basic)
PA_basic_df = spark.createDataFrame(PA_basic)
PA_basic_df.write.mode('overwrite').parquet("hdfs://localhost:9999/user/stock/pa_basic")

     ts_code exchange  reg_capital province city  \
0  601318.SH      SSE  1828024.141       广东  深圳市   

                                        introduction  \
0  公司于1988年诞生于深圳蛇口，是中国第一家股份制保险企业，至今已发展成为融保险、银行、投资...   

                                      business_scope  employees  \
0  (一)投资保险企业；(二)监督管理控股投资企业的各种国内、国际业务；(三)开展保险资金运用业...     372194   

                                   main_business  
0  主营业务:以保险业务为核心,以统一品牌向客户提供包括保险,银行,证券,信托等多元化金融服务  


In [53]:
PA_daily = pro.query('daily', ts_code = '601318.SH')
print(PA_daily)
PA_daily_df = spark.createDataFrame(PA_daily)
PA_daily_df.write.mode('overwrite').parquet("hdfs://localhost:9999/user/stock/pa_daily")

        ts_code trade_date   open   high    low  close  pre_close  change  \
0     601318.SH   20210208  79.73  80.43  78.28  78.57      79.72   -1.15   
1     601318.SH   20210205  80.00  80.28  78.61  79.72      79.62    0.10   
2     601318.SH   20210204  78.00  80.32  77.70  79.62      76.54    3.08   
3     601318.SH   20210203  77.74  77.74  75.30  76.54      78.18   -1.64   
4     601318.SH   20210202  79.28  79.45  77.95  78.18      79.27   -1.09   
...         ...        ...    ...    ...    ...    ...        ...     ...   
3325  601318.SH   20070307  45.29  46.90  44.91  46.48      45.25    1.23   
3326  601318.SH   20070306  44.00  46.00  43.75  45.25      44.38    0.87   
3327  601318.SH   20070305  45.98  46.17  43.53  44.38      46.41   -2.03   
3328  601318.SH   20070302  47.00  47.44  45.88  46.41      46.79   -0.38   
3329  601318.SH   20070301  50.00  50.97  45.80  46.79      33.80   12.99   

      pct_chg         vol        amount  
0     -1.4425   796546.99  6.2869

In [54]:
test = spark.read.parquet("hdfs://localhost:9999/user/stock/pa_daily")
test.show()


+---------+----------+-----+-----+-----+-----+---------+------+-------+----------+--------------+
|  ts_code|trade_date| open| high|  low|close|pre_close|change|pct_chg|       vol|        amount|
+---------+----------+-----+-----+-----+-----+---------+------+-------+----------+--------------+
|601318.SH|  20190529| 77.5|80.98|77.18|79.78|    78.19|  1.59| 2.0335| 968273.16|   7677708.159|
|601318.SH|  20190528|77.44|78.77|77.21|78.19|    77.18|  1.01| 1.3086| 893839.92|   6978721.843|
|601318.SH|  20190527|76.25|77.66|75.35|77.18|     76.5|  0.68| 0.8889| 589731.73|   4515690.456|
|601318.SH|  20190524| 75.9|77.27| 75.8| 76.5|    75.44|  1.06| 1.4051| 512353.15|   3923969.868|
|601318.SH|  20190523| 76.5|76.83|75.05|75.44|    76.54|  -1.1|-1.4372| 645670.76|   4884305.938|
|601318.SH|  20190522|78.22| 78.5| 77.2|77.64|     78.2| -0.56|-0.7161| 516731.93|   4013374.792|
|601318.SH|  20190521|78.05|79.25|77.68| 78.2|    77.87|  0.33| 0.4238| 630197.96|   4945455.898|
|601318.SH|  2019052